## Task 1: Acquire, preprocess, and analyze the data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re

data1 = pd.read_csv('SearchTrendsDataset.csv')
data2 = pd.read_csv('COVIDhospitalizationCasesDataset.csv',index_col = 0)

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### dataset1 

In [2]:
# Are there any symptoms that have no search data available?

missingTotal = data1.isnull().sum().sort_values(ascending=False)
invalidSymptom = missingTotal[missingTotal >= 608]
invalidSymptom.count()

303

In [3]:
# drop symptoms that have no valid data

data1 = data1.dropna(axis=1,how='all')  
data1.shape
data1['date'] = pd.to_datetime(data1['date'])
data1

,open_covid_region_code,country_region_code,country_region,sub_region_1,sub_region_1_code,date,symptom:Adrenal crisis,symptom:Ageusia,symptom:Allergic conjunctivitis,symptom:Amblyopia,...,symptom:Tenderness,symptom:Thyroid nodule,symptom:Trichoptilosis,symptom:Upper respiratory tract infection,symptom:Urethritis,symptom:Urinary urgency,symptom:Vasculitis,symptom:Ventricular fibrillation,symptom:Viral pneumonia,symptom:Yawn
0,US-AK,US,United States,Alaska,US-AK,2020-01-06,12.69,NaN,NaN,NaN,...,14.28,15.86,11.90,18.84,13.88,NaN,14.08,9.91,NaN,14.28
1,US-AK,US,United States,Alaska,US-AK,2020-01-13,9.56,NaN,NaN,NaN,...,13.39,12.24,7.84,16.83,13.01,7.65,15.30,8.41,NaN,16.26
2,US-AK,US,United States,Alaska,US-AK,2020-01-20,NaN,NaN,NaN,NaN,...,12.88,17.66,15.27,18.40,12.14,7.54,10.30,16.19,NaN,17.48
3,US-AK,US,United States,Alaska,US-AK,2020-01-27,15.31,7.47,NaN,NaN,...,13.30,12.76,9.84,20.41,17.49,NaN,11.30,NaN,NaN,10.93
4,US-AK,US,United States,Alaska,US-AK,2020-02-03,8.81,NaN,NaN,NaN,...,11.81,16.68,10.68,20.62,16.87,NaN,14.24,10.68,10.5,18.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,US-WY,US,United States,Wyoming,US-WY,2020-08-24,4.42,3.31,3.42,5.19,...,3.98,3.87,2.26,3.98,4.09,NaN,3.64,2.48,NaN,3.87
604,US-WY,US,United States,Wyoming,US-WY,2020-08-31,5.03,2.79,NaN,5.75,...,3.07,3.07,2.57,2.68,NaN,NaN,5.08,NaN,NaN,3.63
605,US-WY,US,United States,Wyoming,US-WY,2020-09-07,3.36,3.25,2.20,3.63,...,4.29,2.81,NaN,3.19,3.19,NaN,4.51,NaN,NaN,4.02
606,US-WY,US,United States,Wyoming,US-WY,2020-09-14,4.15,4.97,NaN,5.14,...,3.93,5.63,3.06,4.59,3.22,2.68,3.88,NaN,NaN,4.37


In [4]:
# normalize the features in each region using the median of features in each region
import numpy as np
regionList = data1["open_covid_region_code"].drop_duplicates().values.tolist()
features = data1.columns.tolist()
features = features[6:] # features are all symtoms

#x = data1.loc[data1["open_covid_region_code"]=='US-AK', features]
#x.fillna(0, inplace=True)
#symtom_m= x.mean()
#symtom_m
#mean = symtom_m.mean()
#x = pd.DataFrame(x, columns = features)
#x = x.apply(lambda x: x/mean)

df_norm = pd.DataFrame()
# normalize the features in each region, divide values by the median of each region
for region in regionList:
    df_region = data1.loc[data1["open_covid_region_code"]==region, features]
    df_region.fillna(0, inplace=True)
    symtom_m= df_region.mean(axis=1)
    mean = symtom_m.mean()
    df_region = pd.DataFrame(df_region, columns = features)
    df_region = df_region.apply(lambda x: x/mean)
    df_norm = df_norm.append(df_region)

#df_norm = df_norm.round(3)
df_rest = data1.iloc[:,:6]
df_norm.reset_index(drop=True, inplace=True)
data1 = pd.concat([df_rest,df_norm], axis = 1)
data1.to_csv('data1.csv')
data1


,open_covid_region_code,country_region_code,country_region,sub_region_1,sub_region_1_code,date,symptom:Adrenal crisis,symptom:Ageusia,symptom:Allergic conjunctivitis,symptom:Amblyopia,...,symptom:Tenderness,symptom:Thyroid nodule,symptom:Trichoptilosis,symptom:Upper respiratory tract infection,symptom:Urethritis,symptom:Urinary urgency,symptom:Vasculitis,symptom:Ventricular fibrillation,symptom:Viral pneumonia,symptom:Yawn
0,US-AK,US,United States,Alaska,US-AK,2020-01-06,1.402725,0.000000,0.000000,0.000000,...,1.578481,1.753130,1.315400,2.082533,1.534265,0.000000,1.556373,1.095430,0.000000,1.578481
1,US-AK,US,United States,Alaska,US-AK,2020-01-13,1.056742,0.000000,0.000000,0.000000,...,1.480102,1.352983,0.866617,1.860352,1.438097,0.845615,1.691229,0.929623,0.000000,1.797346
2,US-AK,US,United States,Alaska,US-AK,2020-01-20,0.000000,0.000000,0.000000,0.000000,...,1.423728,1.952099,1.687913,2.033897,1.341930,0.833455,1.138540,1.789608,0.000000,1.932202
3,US-AK,US,United States,Alaska,US-AK,2020-01-27,1.692335,0.825718,0.000000,0.000000,...,1.470153,1.410463,1.087692,2.256078,1.933307,0.000000,1.249078,0.000000,0.000000,1.208179
4,US-AK,US,United States,Alaska,US-AK,2020-02-03,0.973838,0.000000,0.000000,0.000000,...,1.305452,1.843771,1.180544,2.279291,1.864774,0.000000,1.574059,1.180544,1.160647,2.092482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,US-WY,US,United States,Wyoming,US-WY,2020-08-24,1.291667,0.967289,0.999434,1.516685,...,1.163084,1.130939,0.660445,1.163084,1.195230,0.000000,1.063725,0.724736,0.000000,1.130939
604,US-WY,US,United States,Wyoming,US-WY,2020-08-31,1.469928,0.815328,0.000000,1.680336,...,0.897153,0.897153,0.751037,0.783182,0.000000,0.000000,1.484540,0.000000,0.000000,1.060803
605,US-WY,US,United States,Wyoming,US-WY,2020-09-07,0.981900,0.949755,0.642911,1.060803,...,1.253676,0.821173,0.000000,0.932221,0.932221,0.000000,1.317968,0.000000,0.000000,1.174774
606,US-WY,US,United States,Wyoming,US-WY,2020-09-14,1.212764,1.452394,0.000000,1.502074,...,1.148473,1.645268,0.894231,1.341346,0.940988,0.783182,1.133861,0.000000,0.000000,1.277055


### dataset2

In [5]:
data2.shape

(96064, 61)

In [6]:
#data2 removing unrelated features and regions

data2 = data2.filter(like='US-', axis=0)
data2 = data2.filter(items=['hospitalized_new','date'], axis=1)
#print data 2
data2



,hospitalized_new,date
open_covid_region_code,,
US-WY,0.0,2020/3/7
US-WY,0.0,2020/3/8
US-WY,0.0,2020/3/9
US-WY,0.0,2020/3/10
US-WY,0.0,2020/3/11
...,...,...
US-AK,0.0,2020/9/27
US-AK,0.0,2020/9/28
US-AK,0.0,2020/9/29


In [7]:
# drop the rows where column hospitalized_new are all NaN

dropInvalid2 = data2.dropna(axis=0,subset=['hospitalized_new'],how='all')
dropInvalid2.shape
dropInvalid2
#all data in US is good.

,hospitalized_new,date
open_covid_region_code,,
US-WY,0.0,2020/3/7
US-WY,0.0,2020/3/8
US-WY,0.0,2020/3/9
US-WY,0.0,2020/3/10
US-WY,0.0,2020/3/11
...,...,...
US-AK,0.0,2020/9/27
US-AK,0.0,2020/9/28
US-AK,0.0,2020/9/29


In [8]:
# turn index open-region-covid back into col
# dropInvalid2['open_covid_region_code'] = dropInvalid2.index,,,,,,,not needed anymore
dropInvalid2.reset_index(level=0, inplace=True)
dropInvalid2

,open_covid_region_code,hospitalized_new,date
0,US-WY,0.0,2020/3/7
1,US-WY,0.0,2020/3/8
2,US-WY,0.0,2020/3/9
3,US-WY,0.0,2020/3/10
4,US-WY,0.0,2020/3/11
...,...,...,...
11853,US-AK,0.0,2020/9/27
11854,US-AK,0.0,2020/9/28
11855,US-AK,0.0,2020/9/29
11856,US-AK,0.0,2020/9/30


In [9]:
# turn index open-region-covid back into col
# dropInvalid2['open_covid_region_code'] = dropInvalid2.index,,,,,,,not needed anymore
dropInvalid2.reset_index(level=0, inplace=True)
dropInvalid2

,index,open_covid_region_code,hospitalized_new,date
0,0,US-WY,0.0,2020/3/7
1,1,US-WY,0.0,2020/3/8
2,2,US-WY,0.0,2020/3/9
3,3,US-WY,0.0,2020/3/10
4,4,US-WY,0.0,2020/3/11
...,...,...,...,...
11853,11853,US-AK,0.0,2020/9/27
11854,11854,US-AK,0.0,2020/9/28
11855,11855,US-AK,0.0,2020/9/29
11856,11856,US-AK,0.0,2020/9/30


In [10]:
#now set index to be date

preWeeklyData2 = dropInvalid2.set_index('date',drop=False)
#preWeeklyData2 = dropInvalid2
preWeeklyData2['date'] = pd.to_datetime(preWeeklyData2['date'])
preWeeklyData2

,index,open_covid_region_code,hospitalized_new,date
date,,,,
2020/3/7,0,US-WY,0.0,2020-03-07
2020/3/8,1,US-WY,0.0,2020-03-08
2020/3/9,2,US-WY,0.0,2020-03-09
2020/3/10,3,US-WY,0.0,2020-03-10
2020/3/11,4,US-WY,0.0,2020-03-11
...,...,...,...,...
2020/9/27,11853,US-AK,0.0,2020-09-27
2020/9/28,11854,US-AK,0.0,2020-09-28
2020/9/29,11855,US-AK,0.0,2020-09-29


In [11]:
preWeeklyData2.dtypes

index                              int64
open_covid_region_code            object
hospitalized_new                 float64
date                      datetime64[ns]
dtype: object

In [12]:
#sum over the weekly resolution
weeklyData2 = preWeeklyData2.set_index('date').groupby('open_covid_region_code')

#weeklyData2 = weeklyData2.groupby(pd.Grouper(key='date',freq='W')).sum()
#weeklyData2 = preWeeklyData2.groupby(pd.Grouper(freq='W')).sum()
##weeklyData2 = preWeeklyData2.groupby(['date','open_covid_region_code']).sum()
weeklyData2copy = weeklyData2

In [13]:
weeklyData2copy1 = weeklyData2copy.resample('7D',label='left',origin='2020-01-06').sum()
weeklyData2copy1

index  hospitalized_new
open_covid_region_code date                               
US-AK                  2020-03-02  34947               0.0
                       2020-03-09  81578               0.0
                       2020-03-16  81627               0.0
                       2020-03-23  81676               6.0
                       2020-03-30  81725              14.0
...                                  ...               ...
US-WY                  2020-08-31   1260               4.0
                       2020-09-07   1309               9.0
                       2020-09-14   1358              15.0
                       2020-09-21   1407              19.0
                       2020-09-28    826              12.0

[1744 rows x 2 columns]

In [14]:
# put each region back to its row 
finalData2 = weeklyData2copy1.reset_index()

In [15]:
mergedData = data1.merge(finalData2,how='outer')

In [16]:
#drop invalid data point (missing either hospitalization or symptoms)
mergedDatacopy = mergedData 
finalMergedData= mergedDatacopy[:608]
finalMergedData = finalMergedData.dropna(axis=0,subset=['hospitalized_new'],how='all')
finalMergedData

,open_covid_region_code,country_region_code,country_region,sub_region_1,sub_region_1_code,date,symptom:Adrenal crisis,symptom:Ageusia,symptom:Allergic conjunctivitis,symptom:Amblyopia,...,symptom:Trichoptilosis,symptom:Upper respiratory tract infection,symptom:Urethritis,symptom:Urinary urgency,symptom:Vasculitis,symptom:Ventricular fibrillation,symptom:Viral pneumonia,symptom:Yawn,index,hospitalized_new
8,US-AK,US,United States,Alaska,US-AK,2020-03-02,1.616063,0.000000,0.000000,0.000000,...,1.432571,2.405304,1.247972,0.000000,1.469048,1.156226,1.560794,1.028002,34947.0,0.0
9,US-AK,US,United States,Alaska,US-AK,2020-03-09,1.171701,0.000000,1.283344,0.000000,...,1.544214,5.189752,1.135224,0.000000,1.023581,0.000000,2.083639,1.190493,81578.0,0.0
10,US-AK,US,United States,Alaska,US-AK,2020-03-16,1.292188,0.000000,0.706337,0.000000,...,0.999262,8.700434,1.326454,0.000000,1.274501,0.000000,2.756814,1.585113,81627.0,0.0
11,US-AK,US,United States,Alaska,US-AK,2020-03-23,1.232497,1.831612,0.000000,0.000000,...,0.924096,4.149591,0.972733,0.000000,0.746131,0.778186,1.524317,1.734339,81676.0,6.0
12,US-AK,US,United States,Alaska,US-AK,2020-03-30,0.990419,0.990419,0.785924,0.000000,...,1.634855,2.703756,1.084376,0.879881,0.833455,0.785924,0.974944,1.948782,81725.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,US-WY,US,United States,Wyoming,US-WY,2020-08-24,1.291667,0.967289,0.999434,1.516685,...,0.660445,1.163084,1.195230,0.000000,1.063725,0.724736,0.000000,1.130939,1211.0,8.0
604,US-WY,US,United States,Wyoming,US-WY,2020-08-31,1.469928,0.815328,0.000000,1.680336,...,0.751037,0.783182,0.000000,0.000000,1.484540,0.000000,0.000000,1.060803,1260.0,4.0
605,US-WY,US,United States,Wyoming,US-WY,2020-09-07,0.981900,0.949755,0.642911,1.060803,...,0.000000,0.932221,0.932221,0.000000,1.317968,0.000000,0.000000,1.174774,1309.0,9.0
606,US-WY,US,United States,Wyoming,US-WY,2020-09-14,1.212764,1.452394,0.000000,1.502074,...,0.894231,1.341346,0.940988,0.783182,1.133861,0.000000,0.000000,1.277055,1358.0,15.0


In [17]:
finalMergedData.to_csv('merged.csv')